# 10k 샘플 시뮬

In [1]:
import os

# 현재 작업 디렉토리 확인
print("현재 작업 디렉토리:", os.getcwd())

# 노트북이 위치한 디렉토리에서 프로젝트 루트 디렉토리로 이동
# 'notebooks/EDA' 에서 'AI_Lab/computer-vision-competition-1SEN' 으로 이동
os.chdir("../../") 

# 작업 디렉토리 변경 후 다시 확인
print("변경 후 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN/notebooks/EDA
변경 후 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [2]:
import numpy as np, pandas as pd, collections        # 수치 연산, 데이터프레임 처리, 카운터
import json                                          # JSON 파일 처리
from torch.utils.data import DataLoader              # 데이터 배치 로더
from src.data.samplers import make_weighted_sampler  # 클래스 불균형 보정 샘플러 생성 함수
from src.data.dataset import DocTypeDataset          # 커스텀 데이터셋 클래스

CFG = json.load(open("configs/data_config.json"))    # 데이터 설정 JSON 로드

# WeightedRandomSampler 생성
sampler = make_weighted_sampler(
    CFG["paths"]["train_csv"],                       # 학습 CSV 경로
    "data/processed/class_weights.json",             # 클래스 가중치 JSON 경로
    CFG["sampler"]["clip_min"],                      # 최소 클리핑 값
    CFG["sampler"]["clip_max"]                       # 최대 클리핑 값
)

# 학습 데이터셋 인스턴스 생성
ds = DocTypeDataset(
    CFG["paths"]["train_csv"],                       # 학습 CSV 경로
    CFG["paths"]["train_root"],                      # 학습 이미지 루트
    mode="train"                                     # 모드(train)
)

# DataLoader 인스턴스 생성 (샘플러 적용)
dl = DataLoader(
    ds,                                              # 데이터셋 객체
    batch_size=1,                                    # 배치 크기(1개 단위)
    sampler=sampler                                  # WeightedRandomSampler 적용
)

cnt = collections.Counter()                          # 클래스 카운터 초기화

# 10,000번 샘플링 반복
for i, (x, y) in zip(range(10_000), dl):
    cnt[int(y)] += 1                                 # 샘플 라벨 카운트

print(cnt)                                           # 클래스별 샘플링 횟수 출력

# 편차 계산
total = sum(cnt.values())                            # 총 샘플링 수
pct = {k: v/total*100 for k,v in cnt.items()}        # 클래스별 비율 계산
print({k: round(v,2) for k,v in pct.items()})        # 소수점 둘째 자리까지 반올림해 출력

Counter({10: 108, 11: 107, 12: 106, 14: 104, 13: 104, 3: 99, 16: 96, 6: 95, 15: 93, 0: 89, 1: 88, 4: 87, 5: 86, 8: 84, 9: 81, 2: 77, 7: 66})
{3: 6.31, 16: 6.11, 11: 6.82, 8: 5.35, 2: 4.9, 5: 5.48, 14: 6.62, 7: 4.2, 4: 5.54, 0: 5.67, 12: 6.75, 9: 5.16, 10: 6.88, 6: 6.05, 15: 5.92, 1: 5.61, 13: 6.62}
